优化算法主要是通过改变模型内部参数，来最小化(或最大化)损失函数。

模型内部有些参数，是用来计算测试集中目标 Y 的真实值和预测值的偏差程度的，基于这些参数，就形成了损失函数 E(x)。

优化算法分为两大类：  

一阶优化算法：这种算法使用各参数的梯度值来最小化或最大化损失函数 E(x)。最常用的一阶优化算法是梯度下降。二阶优化算法

二阶优化算法：使用了二阶导数(也叫做Hessian方法)来最小化或最大化损失函数，是基于牛顿法计算的。由于二阶导数的计算成本很高，所以这种方法并没有广泛使用。

## 1 SGD  

batch gradient descent 需要每次迭代都需要计算所有训练集数据的梯度,随着数据集的增大,运行速度会越来越慢，目前基本不使用这种优化方法。  

SGD 指 mini-batch stochastic gradient descent 每次迭代计算 mini-batch 的梯度，然后对参数进行更新，是目前最常见的优化方法。  

具体实现:   

$dw_t = \nabla_{w_{t-1}}{f(w_{t-1})}$  

$\Delta{dw_t} = -\eta*dw_t$  

$w_t = w_{t-1} + \Delta{dw_t}$

其中，$\eta$ 是学习率，$dw_t$ 是当前 batch 的梯度， $\eta$ 是学习率。  

优点：  

训练速度快,对于很大的数据集,也能够以较快的速度收敛。

缺点：  

1 SGD 基于 mini-batch 计算梯度，每次迭代梯度受当前 mini-batch 中的数据影响较大，不能真实反映真实的数据梯度，得到的梯度肯定有误差，需要逐渐减小学习率，否则模型不容易收敛，实际应用中,一般对学习率进行线性衰减。 

2 不容易选择合适的学习率。  

3 容易收敛到局部最优。 

## 2 Momentum  

SGD 每次迭代计算的梯度含有比较大的噪音。而 Momentum 方法可以比较好的缓解这个问题。Momentum 借用了物理中的动量概念,积累之前的梯度来替代真正的梯度，即前几次的梯度也会参与运算。  

具体实现:   

$dw_t = \nabla_{w_{t-1}}{f(w_{t-1})}$  

$v_t = {mu} * v_{t-1} -\eta * dw_t$   

$ w_t = w_{t-1} + v_t$  
 
其中，$dw_t$ 是当前 batch 的梯度，$v_t$ 是用 Momentum 方法计算出来要更新的梯度大小，${mu} \ 和 \  \eta$ 分别是 Momentum 和 学习率。  
一般 $\mu$ 取值有0.5, 0.9, 0.99 等。也可以让 $\mu$ 随时间而变化,一开始小点,后来再加大。

优点：  

当前梯度方向与之前梯度方向一致时,能够加速学习  

当前梯度方向与之前梯度方向不一致时,能够抑制震荡，并且还有一定摆脱局部最优的能力


## 3 Nesterov Momentum  

这是对之前的 Momentum 的一种改进,在理论上对于凸函数它能得到更好的收敛，在实践中也确实比标准动量表现更好一些。  

Nesterov 动量的核心思路是，当参数向量位于某个位置 $w_{t−1}$ 时，观察上面的动量更新公式可以发现，动量部分会通过 ${mu} * v_{t-1}$ 稍微改变参数向量。因此，如果要计算梯度，那么可以将未来的近似位置 $w_{t−1} + {mu} * v_{t-1}$ 看做是要计算梯度的位置，而不是计算之前位置 $w_{t−1}$ 的梯度。  

具体实现:  

$dw_t = \nabla_{w_{t-1}}{f(w_{t-1} + {mu} * v_{t-1}))}$  // 不是计算 $w_{t-1}$ 处的梯度

$v_t = {mu} * v_{t-1} -\eta * dw_t$ 

$ w_t = w_{t-1} + v_t$

然而在实践中，人们更喜欢和普通 SGD 或上面的动量方法一样简单的表达式，通过公式推导，常用的更新公式如下所示：  

$v_{t-1} = v_t$  

$v_t = {mu} * v_{t-1} -\eta * dw_t$  

$w_t = w_{t-1} + (1 + mu) * v - {mu} * v_{t-1} $  

优点：  

Nesterov 算法相对于 Momentum 的改进在于，以“向前看”看到的梯度而不是当前位置梯度去更新。经过变换之后的等效形式中，Nesterov 算法相对于 Momentum 多了一个本次梯度相对上次梯度的变化量，这个变化量本质上是对目标函数二阶导的近似。由于利用了二阶导的信息，Nesterov 算法才会比 Momentum 具有更快的收敛速度。

## 4 Adagrad  

Adagrad是一个由 Duchi 等提出的适应性学习率算法,只是需要设定一个全局的学习速率,就可以自动变更学习速率。  

具体实现：  

$dw_t = \nabla_{w_{t-1}}{f(w_{t-1})}$  

$n_t = n_{t-1} + {dw_t}^2$ 

$ w_t = w_{t-1} - \frac {\eta}{\sqrt {n_t + \epsilon}} * dw_t$  


优点：  

能够实现学习率的自动调整。如果这次梯度大,那么学习速率衰减的就快一些;如果这次梯度小,那么学习速率衰减的就慢一些。

缺点：  

1 仍依赖于人工设置一个全局学习率  

2 在深度学习中，分母上梯度平方的累加将会越来越大，学习率单调变小，会造成训练提前结束。

## 5 RMSProp  

RMSProp 是一个非常高效，但没有公开发表的适应性学习率方法。这个方法用一种很简单的方式修改了 Adagrad 方法，让它不那么激进,使用了一个梯度平方的滑动平均：  

具体实现：  

$dw_t = \nabla_{w_{t-1}}{f(w_{t-1})}$  

$n_t = decay * n_{t-1} + (1 - decay) * {dw_t}^2$ 

$ w_t = w_{t-1} - \frac {\eta}{\sqrt {n_t} + \epsilon} * dw_t$  

decay 是一个超参数，常用的值是[0.9,0.99,0.999], 因此，RMSProp 仍然是基于梯度的大小来对每个权重的学习率进行修改，但是和 Adagrad 不同，其更新不会让学习率单调变小。  

优点：  

1 RMSprop 是 Adagrad 的改进，解决了学习率单调变小，训练提前结束的问题  
2 适合处理非平稳目标 - 对于RNN效果很好

缺点：  

仍依赖于人工设置一个全局学习率  

## 6 Adam  

Adam(Adaptive Moment Estimation)本质上是带有动量项的RMSprop，它利用梯度的一阶矩估计和二阶矩估计动态调整每个参数的学习率。  

具体实现：  

$dw_t = \nabla_{w_{t-1}}{f(w_{t-1})}$  

$n_t = decay * n_{t-1} + (1 - decay) * {dw_t}^2$ 

$m_t = mu * m_{t-1} + (1 - mu) * {dw_t}^2$

$ w_t = w_{t-1} - \frac {\eta}{\sqrt {n_t} + \epsilon} * m_t$  

这个更新方法看起来和 RMSProp 很像，除了使用的是平滑版的梯度 m，而不是用的原始梯度向量 $dw_t$。论文中推荐的参数值 eps = 1e-8, mu = 0.9, decay = 0.999。在实际操作中，推荐 Adam 作为默认的算法，一般而言跑起来比 RMSProp 要好一点。但是也可以试试 SGD+Nesterov 动量。  

优点：  

1 结合了Adagrad 善于处理稀疏梯度和 RMSprop 善于处理非平稳目标的优点  

2 对内存需求较小  

3 为不同的参数计算不同的自适应学习率  

4 也适用于大多非凸优化 - 适用于大数据集和高维空间

## 学习率退火

在训练深度网络的时候，让学习率随着时间退火通常是有帮助的。可以这样理解：如果学习率很高，系统的动能就过大，参数向量就会无规律地跳动，不能够稳定到损失函数更深更窄的部分去。  

知道什么时候开始衰减学习率是有技巧的：慢慢减小它，可能在很长时间内只能是浪费计算资源地看着它混沌地跳动，实际进展很少。但如果快速地减少它，系统可能过快地失去能量，不能到达原本可以到达的最好位置。通常，实现学习率退火有3种方式：  

随步数衰减：每进行几个周期就根据一些因素降低学习率，典型的值是每过 5 个周期就将学习率减少一半，或者每 20 个周期减少到之前的 0.1。这些数值的设定是严重依赖具体问题和模型的选择的。在实践中可能看见这么一种经验做法：使用一个固定的学习率来进行训练的同时观察验证集错误率，每当验证集错误率停止下降，就乘以一个常数（比如0.5）来降低学习率。  

指数衰减: 数学公式是 $\alpha=\alpha_0e^{-kt}$，其中 $\alpha_0$, k 是超参数，t 是迭代次数  

1/t 衰减: 数学公式是 $\alpha=\alpha_0/(1+kt)$，其中 $\alpha_0$ ,k 是超参数，t是迭代次数。  

在实践中，我们发现随步数衰减更受欢迎，因为它使用的超参数比 k 更有解释性。最后，如果有足够的计算资源，可以让衰减更加缓慢一些，让训练时间更长些。

## 结论

SGD通常训练时间更长，但是在好的初始化和学习率调度方案的情况下，结果更可靠  

如果在意更快的收敛，并且需要训练较深较复杂的网络时，推荐使用学习率自适应的优化方法。  

推荐的两个更新方法是SGD+Nesterov动量方法，或者Adam方法。

## 参考  

1 [CS231n课程笔记翻译](https://zhuanlan.zhihu.com/p/21798784?refer=intelligentunit)  
2 [深度学习最全优化方法总结比较](https://zhuanlan.zhihu.com/p/22252270)  